In [2]:
import requests
import base64
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# GPT-4 Vision API endpoint
API_ENDPOINT = "https://api.openai.com/v1/chat/completions"

# Your API key (store this securely, preferably as an environment variable)
API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def classify_image(image_path, prompt):
    # Encode the image
    base64_image = encode_image(image_path)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }

    payload = {
        "model": "gpt-4-vision-preview",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 300
    }

    response = requests.post(API_ENDPOINT, headers=headers, json=payload)
    return response.json()

In [4]:
def process_folder(folder_path, prompt, output_file):
    folder = Path(folder_path)
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif']

    with open(output_file, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Image', 'Classification'])

        for image_path in folder.iterdir():
            if image_path.suffix.lower() in image_extensions:
                print(f"Processing {image_path.name}...")
                result = classify_image(str(image_path), prompt)
                classification = result['choices'][0]['message']['content']
                csvwriter.writerow([image_path.name, classification])
                print(f"Classified as: {classification}")

In [1]:


# Example usage
folder_path = "path/to/your/image/folder"
output_file = "classification_results.csv"
prompt = """
Analysiere das Bild und klassifiziere es in eine der folgenden Kategorien:

1. Heizraum
2. Heizkörper
3. Hahnblock in Nahaufnahme
4. Keines der obigen

Wichtige Anweisungen:
- Wähle die Kategorie, die am besten passt.
- Antworte ausschließlich mit der entsprechenden Nummer (1, 2, 3 oder 4).
- Gib keine zusätzlichen Erklärungen oder Texte aus.
"""

process_folder(folder_path, prompt, output_file)
print(f"Classification complete. Results saved to {output_file}")

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/your/image.jpg'